In [8]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import re
import os
import base64
import datetime
import warnings
warnings.filterwarnings('ignore')
import pymssql

In [160]:
def get_tick_data(tick):
    df = pd.DataFrame()
    try:
        error = 0
        period_start = 0
        perid_end = 9999999999
        df = pd.read_csv(f'https://query1.finance.yahoo.com/v7/finance/download/{tick}?period1={period_start}&period2={perid_end}&interval=1d&events=history&includeAdjustedClose=true')
    except:
        error = 1
        print('check url and ticks')
    if error == 0:
        df.to_csv(f'{tick}.csv')
        return df

In [165]:
def load_tick(tick, server, user, password, db, tableName):
    df = get_tick_data(tick)
    #clean up the df, make it same structure as the target table
    df.insert(0, 'tick', tick) # index, colName, value
    currTime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    df['Insert_Date'] = currTime
    #get the column name
    cols = list(df.columns)
    for i in range(len(cols)):
        cols[i] = str(cols[i]).replace(' ','_')
    
    df = df.to_numpy()
    
    command = []
    for r in df:
        r = list(r) + [currTime]
        curr = f"insert into {tableName} values('"
        for i in range(len(cols)):
            curr+= str(r[i]) + "','"
        curr = curr[:-2]
        curr+=")"
        command.append(curr)
    
    # execute the insert 
    conn = pymssql.connect(server, user, password, db)
    cursor = conn.cursor()
    cnt = 0
    for c in command:
        cnt += 1 
        cursor.execute(c)
        if cnt % 1000 == 0:
            print(cnt)
    conn.commit()
    conn.close()
    print(f'{cnt} records inserted')

In [175]:
server = "Xxxxxxx"
user = "xxxxxx"
password = "xxxxxxx"
db = "Test"
tableName = "market"
ticks = ['bidu']

In [176]:
for tick in ticks:
    print(f'Loading: {tick}')
    load_tick(tick.upper(), server, user, password, db, tableName)

Loading: bidu
1000
2000
3000
3900 records inserted
